In [1]:
import sys
sys.path.append('..')
from src.utilities import *
from src.models.train_model import train_model
from src.models.param_opt import bayes_parameter_opt_lgb

## 3. Modelling

#### 3.1 Load master table

In [2]:
master = pd.read_csv(os.path.join(processed_path, 'master.csv'))
master.head()

,month,region,brand,sales,sales_univ_b1_1mo,sales_univ_b12_market_1mo,sales_univ_b2_1mo,sales_univ_b3_1mo,sales_univ_b3_market_1mo,sales_univ_1mo,sales_univ_market_1mo,sales_univ_brand_1mo,sales_region_b3_1mo,sales_region_b3market_1mo,sales_region_b12market_1mo,sales_univ_b1_2mo,sales_univ_b1_3mo,sales_univ_b1_4mo,sales_univ_b1_trend_2mo,sales_univ_b1_trend_3mo,sales_univ_b1_trend_4mo,sales_univ_b12_market_2mo,sales_univ_b12_market_3mo,sales_univ_b12_market_4mo,sales_univ_b12_market_trend_2mo,sales_univ_b12_market_trend_3mo,sales_univ_b12_market_trend_4mo,sales_univ_b2_2mo,sales_univ_b2_3mo,sales_univ_b2_4mo,sales_univ_b2_trend_2mo,sales_univ_b2_trend_3mo,sales_univ_b2_trend_4mo,sales_univ_b3_2mo,sales_univ_b3_3mo,sales_univ_b3_4mo,sales_univ_b3_trend_2mo,sales_univ_b3_trend_3mo,sales_univ_b3_trend_4mo,sales_univ_b3_market_2mo,sales_univ_b3_market_3mo,sales_univ_b3_market_4mo,sales_univ_b3_market_trend_2mo,sales_univ_b3_market_trend_3mo,sales_univ_b3_market_trend_4mo,sales_univ_2mo,sales_univ_3mo,sales_univ_4mo,sales_univ_trend_2mo,sales_univ_trend_3mo,sales_univ_trend_4mo,sales_univ_market_2mo,sales_univ_market_3mo,sales_univ_market_4mo,sales_univ_market_trend_2mo,sales_univ_market_trend_3mo,sales_univ_market_trend_4mo,sales_univ_brand_2mo,sales_univ_brand_3mo,sales_univ_brand_4mo,sales_univ_brand_trend_2mo,sales_univ_brand_trend_3mo,sales_univ_brand_trend_4mo,sales_region_b3_2mo,sales_region_b3_3mo,sales_region_b3_4mo,sales_region_b3_trend_2mo,sales_region_b3_trend_3mo,sales_region_b3_trend_4mo,sales_region_b3market_2mo,sales_region_b3market_3mo,sales_region_b3market_4mo,sales_region_b3market_trend_2mo,sales_region_b3market_trend_3mo,sales_region_b3market_trend_4mo,sales_region_b12market_2mo,sales_region_b12market_3mo,sales_region_b12market_4mo,sales_region_b12market_trend_2mo,sales_region_b12market_trend_3mo,sales_region_b12market_trend_4mo
0,2020-01,region_0,brand_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01,region_0,brand_2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01,region_1,brand_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01,region_1,brand_2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01,region_2,brand_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3.2 Train / val / test split

In [3]:
x_train = master[(master.sales.isna()==False)].drop(columns = [ 'region', 'brand']).copy()
x_train = x_train[x_train.month >= '2020-06']
x_train['train'] =  (x_train.month <= '2021-04').astype(float)
x_train.drop(columns = 'month', inplace = True)

# y_train = master[(master.sales.isna()==False) & (master.month >= '2020-06') & 
#                  (master.month <= '2021-04')].sales

# x_val = master[(master.sales.isna()==False) & (master.month > '2021-04')
#               ].drop(columns = ['month', 'region', 'brand'])
# y_val = master[(master.sales.isna()==False) & (master.month > '2021-04')
#               ].sales

x_test = master[(master.sales.isna()) & (master.month >= '2020-07')]

### 3.3 Parameter search

In [9]:
# best_params = bayes_parameter_opt_lgb(X = x_train, y = y_train,
#                                       init_round=3, opt_round=2, n_folds=5, random_seed=6, n_estimators=10000, 
#                                       learning_rate=0.01, save_path = '')
# best_params = best_params.max['params']
# best_params

### 3.4 Model training

In [4]:
best_params = {'metric': 'rmse',
               'bagging_fraction': 0.8,
               'feature_fraction': 0.5,
               'lambda_l1': 1,
               'lambda_l2': 1,
               'max_depth': 3,
               'min_child_weight': 5,
               'min_split_gain': 0.1,
               'num_leaves': 6}

model = train_model(x_train, target_name = 'sales', split = '', 
                    model_type = 'lgb', params = best_params, objective = 'regression', 
                    metric = 'rmse', save_path = '')

####################     training with  3322      ####################
####################     validating with  1208      ####################
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002362 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5986
[LightGBM] [Info] Number of data points in the train set: 3322, number of used features: 77
[LightGBM] [Info] Start training from score 1438.125719
[1]	training's rmse: 2230.21	valid_1's rmse: 3471.55
Training until validation scores don't improve for 20 rounds
[2]	training's rmse: 2158.74	valid_1's rmse: 3372.57
[3]	training's rmse: 2095.62	valid_1's rmse: 3292.27
[4]	training's rmse: 2060.46	valid_1's rmse: 3239.35
[5]	training's rmse: 2016.46	valid_1's rmse: 3173.27
[6]	training's rmse: 1975.85	valid_1's rmse: 3090.07
[7]	training's rmse: 1940.8	valid_1's rmse: 3040.5
[8]	training's rmse: 1910.7	valid_1's rmse: 2991.71
[9]	training's rmse: 1884.1

In [6]:
pd.DataFrame({'feature': x_train.drop(columns = ['sales', 'train']).columns, 
              'gain': model.feature_importance()}).sort_values('gain', ascending = False).head(30)

,feature,gain
7,sales_univ_brand_1mo,24
71,sales_region_b12market_2mo,21
60,sales_region_b3_3mo,18
10,sales_region_b12market_1mo,16
72,sales_region_b12market_3mo,13
59,sales_region_b3_2mo,12
66,sales_region_b3market_3mo,11
8,sales_region_b3_1mo,11
53,sales_univ_brand_2mo,10
70,sales_region_b3market_trend_4mo,9


### 3.5 Prediction storage

In [7]:
x_test['sales'] = model.predict(x_test.drop(columns = ['month', 'region', 'brand', 'sales']))
x_test = x_test[['month', 'region', 'brand', 'sales']]
x_test['lower'] = 0
x_test['upper'] = 100
x_test.head()

,month,region,brand,sales,lower,upper
2714,2020-07,region_151,brand_1,134.278035,0,100
2715,2020-07,region_151,brand_2,134.278035,0,100
2716,2020-07,region_152,brand_1,3.668701,0,100
2717,2020-07,region_152,brand_2,3.668701,0,100
2718,2020-07,region_153,brand_1,290.612660,0,100


In [8]:
x_test.to_csv(os.path.join(results_path, 'submission2_team46.csv'),index = False)